In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

Wed Dec 26 2018 

CPython 3.6.6
IPython 6.5.0

numpy 1.15.1
scipy 1.1.0
sklearn 0.19.1
pandas 0.23.4

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.9.0-7-amd64
machine    : x86_64
processor  : 
CPU cores  : 12
interpreter: 64bit


In [2]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nolds
import data
import mne

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
DATA_ROOT = os.path.abspath(os.path.join(PROJ_ROOT, 'data'))
PROCESSED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'processed'))
RAW_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'raw'))
LABELED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'labeled'))
DURATIONS_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'durations'))
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
CHANNEL_NAMES = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
                 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
META_COLUMN_NAMES = ['freq', 'RESP_4W', 'RESP_FIN', 'REMISE_FIN', 'AGE', 'SEX', 'M_1',
       'M_4', 'M_F', 'délka léčby', 'lék 1', 'lék 2', 'lék 3', 'lék 4']
META_FILE_NAME = 'DEP-POOL_Final_144.xlsx'
meta_df = pd.read_excel(os.path.join(RAW_ROOT, META_FILE_NAME), index_col='ID', names=META_COLUMN_NAMES)

raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '1a.fif'))
t = pd.DataFrame(raw_fif.get_data())
data = pd.DataFrame(np.transpose(t.values), columns=CHANNEL_NAMES)
data = np.transpose(data.values)
metapkl = pd.read_pickle(os.path.join(LABELED_ROOT, 'processed', 'meta', 'meta.pkl'))
metapkl

Opening raw data file /home/kovar/thesis_project/data/processed/1a.fif...
This filename (/home/kovar/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 19104 =      0.000 ...    76.416 secs
Ready.


<ipython-input-2-2bb109ef8967>:25: RuntimeWarning: This filename (/home/kovar/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '1a.fif'))


resp b/a sex  age    sc  sc_bef  sc_aft  dep  dep_bef  dep_aft  \
patient trial                                                                  
1       a        1   0   0   49  26.0      26      13    0        0        0   
        b        1   1   0   49  13.0      26      13   -1        0        0   
2       a        0   0   0   36  31.0      31      30    1        1        1   
        b        0   1   0   36  30.0      31      30    1        1        1   
3       a        1   0   0   48  25.0      25       7    0       -1       -1   
        b        1   1   0   48   7.0      25       7   -1       -1       -1   
4       a        0   0   0   55  26.0      26      25    0        0        1   
        b        0   1   0   55  25.0      26      25    0        0        1   
5       a        0   0   0   35  26.0      26      37    0        0        1   
        b        0   1   0   35  37.0      26      37    1        0        1   
6       a        0   0   0   37  31.0      31      17    1        1        0   
        b        0   1   0   37  17.0      31      17   -1        1        0   
7       a        0   0   0   63  30.0      30      25    1        1        1   
        b        0   1   0   63  25.0      30      25    0        1        1   
8       a        1   0   1   50  24.0      24      11    0       -1       -1   
        b        1   1   1   50  11.0      24      11   -1       -1       -1   
9       a        0   0   1   59  34.0      34      27    1        1        1   
        b        0   1   1   59  27.0      34      27    1        1        1   
10      a        1   0   0   49  36.0      36      17    1        1        0   
        b        1   1   0   49  17.0      36      17   -1        1        0   
11      a        0   0   1   53  27.0      27      26    1        0        1   
        b        0   1   1   53  26.0      27      26    0        0        1   
12      a        1   0   1   59  30.0      30      12    1        1       -1   
        b        1   1   1   59  12.0      30      12   -1        1       -1   
13      a        1   0   0   57  26.0      26       7    0        0       -1   
        b        1   1   0   57   7.0      26       7   -1        0       -1   
14      a        0   0   0   30  28.0      28      17    1        0        0   
        b        0   1   0   30  17.0      28      17   -1        0        0   
15      a        1   0   0   35  25.0      25       3    0       -1       -1   
        b        1   1   0   35   3.0      25       3   -1       -1       -1   
...            ...  ..  ..  ...   ...     ...     ...  ...      ...      ...   
119     a        1   0   1   61  23.0      23       4    0       -1       -1   
        b        1   1   1   61   4.0      23       4   -1       -1       -1   
120     a        0   0   0   41  28.0      28      19    1        0        0   
        b        0   1   0   41  19.0      28      19   -1        0        0   
121     a        0   0   0   34  24.0      24      26    0       -1        1   
        b        0   1   0   34  26.0      24      26    0       -1        1   
122     a        0   0   1   55  35.0      35      19    1        1        0   
        b        0   1   1   55  19.0      35      19   -1        1        0   
123     a        0   0   1   54  24.0      24      25    0       -1        1   
        b        0   1   1   54  25.0      24      25    0       -1        1   
124     a        0   0   0   52  30.0      30      23    1        1        1   
        b        0   1   0   52  23.0      30      23    0        1        1   
125     a        1   0   0   56  26.0      26      11    0        0       -1   
        b        1   1   0   56  11.0      26      11   -1        0       -1   
126     a        0   0   0   40  25.0      25      17    0       -1        0   
        b        0   1   0   40  17.0      25      17   -1       -1        0   
127     a        0   0   0   35  25.0      25      21    0       -1        1   
        b     

In [106]:
def f(row, col):
    if row[col] <= 19:
        return -1
    if row[col] <= 34:
        return 0
    else:
        return 1
def ff(row, col, n=5):
    for i in range(n):
        if row[col] <= metapkl.quantile((i+1)/n)[col]:
            return i
    
metapkl['dep'] = metapkl.apply(lambda row: ff(row, 'sc'), axis=1)
metapkl['dep_bef'] = metapkl.apply(lambda row: ff(row, 'sc_bef'), axis=1)
metapkl['dep_aft'] = metapkl.apply(lambda row: ff(row, 'sc_aft'), axis=1)
metapkl = metapkl.astype({'dep': 'category', 'dep_bef': 'category', 'dep_aft': 'category'})                                           
print(metapkl['dep'].value_counts())
print(metapkl['dep_bef'].value_counts())
print(metapkl['dep_aft'].value_counts())

3    58
2    55
0    55
1    54
4    44
Name: dep, dtype: int64
0    100
3     56
2     44
4     40
1     26
Name: dep_bef, dtype: int64
0    68
3    58
2    50
4    48
1    42
Name: dep_aft, dtype: int64


In [8]:
def prepare_dfs(col='lyap', kind='processed'):
    df = pd.read_pickle(os.path.join(LABELED_ROOT, kind, col, f'training_{col}.pkl')).dropna()
    df_bef = pd.read_pickle(os.path.join(LABELED_ROOT, kind, col, f'{col}_bef.pkl')).dropna()
    df_aft = pd.read_pickle(os.path.join(LABELED_ROOT, kind, col, f'{col}_aft.pkl')).dropna()
    return df, df_bef, df_aft
    
def prepare_resp_non(col='lyap'):
    df, df_bef, df_aft = prepare_dfs(col)
    return df[df['change'] >= df.quantile(0.66)['change']], df[df['change'] <= df.quantile(0.33)['change']]

col = 'all'
df, df_bef, df_aft = prepare_dfs(col)
# df_resp, df_non = prepare_resp_non(col)
df

channel               FP1                                            \
feature              lyap        corr       dfa     hurst    sampen   
patient trial                                                         
1       a       0.0354587     1.06838  0.875958  0.754148  0.591666   
        b       0.0399611    0.106497  0.800248  0.793054   0.72875   
2       b       0.0371767    0.634254  0.437548  0.593818  0.620112   
3       a       0.0386179     2.51507  0.341363   0.45207  0.466363   
        b       0.0365023     2.48113  0.406266  0.565702  0.518688   
4       a       0.0375807     2.87517  0.565936  0.549389  0.597395   
        b       0.0363211     1.06902  0.648121  0.657075  0.601317   
5       a       0.0351841    0.434594  0.812759  0.660657  0.652824   
        b       0.0344521    0.459655   0.56224  0.679422  0.632775   
6       b       0.0512405    0.293646  0.626166  0.687912  0.704864   
7       a       0.0520428    0.321962  0.586863  0.662046  0.649313   
        b        0.055538     1.25891  0.497486  0.541302  0.612224   
8       b       0.0320772     2.46557  0.613183  0.619332  0.489485   
9       a       0.0511946    0.303689  0.803456  0.664935  0.672748   
        b        0.036028     1.00812  0.861972  0.656427  0.584781   
10      a       0.0656164   0.0139393  0.688589  0.700691  0.940555   
        b       0.0450889   0.0758747  0.756033  0.701519  0.776302   
11      a       0.0274139    0.104895  0.579021  0.597636  0.771227   
        b       0.0257492   0.0944609   0.77545  0.697739  0.805837   
12      a      0.00846576     2.07888   1.09068  0.917632  0.532951   
        b       0.0313727  0.00724362  0.682067  0.716169  0.888382   
13      b       0.0592675   0.0360323  0.560523  0.683081   0.76922   
14      a       0.0341264    0.189369  0.842003  0.820194  0.709296   
        b       0.0209462    0.618397  0.928748   0.81469  0.611801   
15      a       0.0322275    0.228731  0.905694  0.789846  0.709038   
        b       0.0206867    0.599345  0.893735   0.76889  0.613149   
16      a       0.0330165    0.837145  0.465842  0.597906  0.576177   
        b       0.0314038    0.612233  0.670771  0.670839  0.618317   
17      a       0.0642885   0.0528201  0.454658   0.56979  0.787714   
        b       0.0431283   0.0412609  0.756309  0.630051  0.807832   
...                   ...         ...       ...       ...       ...   
117     a       0.0255894     0.19085  0.819095  0.737924  0.734178   
        b       0.0440871   0.0922706  0.570014  0.692036  0.778484   
118     a       0.0355514    0.017817  0.639167  0.638522  0.875469   
119     a       0.0359397    0.473043  0.889105  0.752552  0.667802   
        b       0.0281931    0.165703  0.820736  0.767454  0.736928   
120     a       0.0270263    0.080314  0.414707  0.589457  0.775412   
121     a       0.0202561    0.896134  0.940726  0.811045  0.587278   
        b       0.0584668   0.0131312  0.536531  0.637635   1.00172   
122     a       0.0279221     0.21726   0.83934  0.618311  0.680467   
        b       0.0267358    0.184354  0.549117  0.580665  0.704016   
123     a       0.0360139    0.212727  0.720932  0.683454  0.697911   
        b       0.0447839    0.153503   0.70432  0.626479  0.719974   
124     a       0.0471553    0.249413   0.66091  0.664069  0.698469   
        b       0.0422593   0.0739531  0.705418  0.674528  0.753907   
125     a       0.0483008    0.105811  0.721033  0.666663  0.742761   
126     a       0.0435048   0.0341872   0.68984  0.699721  0.813593   
        b       0.0489387    0.180365  0.656238  0.738625  0.712423   
127     a       0.0311236    0.130674  0.670575  0.690279   0.71524   
        b       0.0430605   0.0351511   0.71863  0.697897   0.78813   
128     a       0.0433045   0.0447943  0.766633  0.709656  0.849587   
129     a       0.0488494    0.169712  0.773959  0.717622  0.694048   
        b       0.0449091   0.0675647  0.668445  0.705393  0.787297   
130     a       0.038655

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import (SelectFromModel, RFE, RFECV, SelectKBest, mutual_info_classif)
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, grid_search, datasets
from genetic_selection import GeneticSelectionCV
from functools import wraps
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

def print_params(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        print(f'{args} {kwargs}')
        res = f(*args, **kwargs)
        return res
    return wrapper

@print_params
def predict(lab, ba, cols, selector):
    df, df_bef, df_aft = prepare_dfs('all')
    df = df.loc[(slice(None), slice(ba)), (CHANNEL_NAMES, (cols))]
    X = df
    X = X.drop(metapkl.columns, axis=1, errors='ignore')
    y = X.join(metapkl)[lab]
    
    X_train, X_test, y_train, y_test = \
        train_test_split(
            X, y, test_size=0.3, random_state=213)
    
    # unique, counts = np.unique(y_train, return_counts=True) 
    # print(counts[0] / (counts[0] + counts[1]))
    unique, counts = np.unique(y_train, return_counts=True)
    print(dict(zip(unique, counts)))
    unique, counts = np.unique(y_test, return_counts=True)
    print(dict(zip(unique, counts)))

    y_train = y_train.astype('int')

    selector.fit(X_train, y_train)
    y_pred = selector.predict(X_test)
    y_pred = y_pred.astype('int')
    y_test = y_test.astype('int')

    print("Accuracy score: %.2f" % metrics.accuracy_score(y_test, y_pred))
    print('Confusion matrix:\n ', metrics.confusion_matrix(y_test, y_pred))
    print('Precision score: ', metrics.precision_score(y_test, y_pred, average='weighted'))
    print('Recall score: ', metrics.recall_score(y_test, y_pred, average='weighted'))
    print('f1 score: ', metrics.f1_score(y_test, y_pred, average='weighted'))
    # print('Coefficients: \n', np.array(X.stack().columns)[selector.support_])

In [15]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return (metrics.f1_score(y, y_pred, average='weighted') 
            + metrics.precision_score(y, y_pred, average='weighted') 
            + metrics.recall_score(y, y_pred, average='weighted') 
            + metrics.accuracy_score(y, y_pred))

estimator_svc = grid_search.GridSearchCV(svm.SVC(class_weight='balanced'), {
    'C': [0.02, 0.5, 1.0, 1.5, 2],
    # 'kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
    'shrinking': (True, False),
    # 'decision_function_shape' : ('ovo', 'ovr'),
}, iid=False, scoring=scorer)

estimator_lg = grid_search.GridSearchCV(LogisticRegression(class_weight='balanced'), {
    'C': [0.5, 1.0, 1.5, 2, 2.5, 3, 3.5],
    'penalty': ['l2', 'l1'],
}, iid=False, scoring=scorer)

estimators = {
    svm.SVC(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 1.5, 2],
        # 'kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
        'shrinking': (True, False),
        # 'decision_function_shape' : ('ovo', 'ovr'),
    },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.5, 1.0, 1.5, 2, 2.5, 3, 3.5],
        'penalty': ['l2', 'l1'],
    }
}

cols = ('lyap', 'corr', 'dfa', 'sampen')

for estimator, params in estimators.items():
    for cols in (('lyap', 'corr', 'dfa', 'sampen'), ('lyap'), ('corr'), ('dfa'), ('sampen')):
        for selector in (RFECV(estimator), GeneticSelectionCV(estimator,
                                          cv=3,
                                          verbose=0,
                                          scoring=scorer,
                                          n_population=50,
                                          crossover_proba=0.8,
                                          mutation_proba=0.2,
                                          n_generations=50,
                                          crossover_independent_proba=0.5,
                                          mutation_independent_proba=0.05,
                                          tournament_size=3,
                                          caching=True,
                                          n_jobs=-1)):
            selector = grid_search.GridSearchCV(selector, params, iid=False, scoring=scorer)
            # predict('dep', None, cols, selector)
            predict('dep_bef', 'a', cols, selector.estimator.estimator)
            # predict('dep_aft', 'b', cols, selector)
            # predict('resp', 'b', cols, selector)

('dep_bef', 'a', ('lyap', 'corr', 'dfa', 'sampen'), SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)) {}
{-1: 31, 0: 28, 1: 25}
{-1: 14, 0: 14, 1: 8}
Accuracy score: 0.33
Confusion matrix:
  [[ 1 10  3]
 [ 4  8  2]
 [ 1  4  3]]
Precision score:  0.2895622895622896
Recall score:  0.3333333333333333
f1 score:  0.29506172839506173
('dep_bef', 'a', ('lyap', 'corr', 'dfa', 'sampen'), SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)) {}
{-1: 31, 0: 28, 1: 25}
{-1: 14, 0: 14, 1: 8}
Accuracy score: 0.33
Confusion matrix:
  [[ 1 10  3]
 [ 4  8  2]
 [ 1  4  3]]
Precision score:  0.2895622895622896
Recall score:  0.3333333333333333
f1 scor

{-1: 31, 0: 28, 1: 25}
{-1: 14, 0: 14, 1: 8}
Accuracy score: 0.56
Confusion matrix:
  [[10  4  0]
 [ 5  8  1]
 [ 3  3  2]]
Precision score:  0.571604938271605
Recall score:  0.5555555555555556
f1 score:  0.5384230233368165
('dep_bef', 'a', 'sampen', LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)) {}
{-1: 31, 0: 28, 1: 25}
{-1: 14, 0: 14, 1: 8}
Accuracy score: 0.36
Confusion matrix:
  [[7 7 0]
 [8 4 2]
 [5 1 2]]
Precision score:  0.3768518518518518
Recall score:  0.3611111111111111
f1 score:  0.3538629126864421
('dep_bef', 'a', 'sampen', LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol